In [2]:
# HEX to Binary conversion
def hex_to_bin(hex_value):
    scale = 16  # base
    num_of_bits = 4 * len(hex_value)
    return bin(int(hex_value, scale))[2:].zfill(num_of_bits)

# Binary to HEX conversion
def bin_to_hex(bin_value):
    scale = 2
    num_of_hex = len(bin_value) // 4
    return hex(int(bin_value, scale))[2:].upper().zfill(num_of_hex)


# Permutation function
def permute(bits, permutation_table):
    return ''.join([bits[i - 1] for i in permutation_table])

# Left shift function
def shift_left(bits, n_shifts):
    return bits[n_shifts:] + bits[:n_shifts]

# XOR operation
def xor(bits1, bits2):
    return ''.join(['0' if bit1 == bit2 else '1' for bit1, bit2 in zip(bits1, bits2)])

# S-box substitution
def sbox_substitution(bits):
    sbox = [
        [[14,4,13,1,2,15,11,8,3,10,6,12,5,9,0,7],
         [0,15,7,4,14,2,13,1,10,6,12,11,9,5,3,8],
         [4,1,14,8,13,6,2,11,15,12,9,7,3,10,5,0],
         [15,12,8,2,4,9,1,7,5,11,3,14,10,0,6,13]],

        [[15,1,8,14,6,11,3,4,9,7,2,13,12,0,5,10],
         [3,13,4,7,15,2,8,14,12,0,1,10,6,9,11,5],
         [0,14,7,11,10,4,13,1,5,8,12,6,9,3,2,15],
         [13,8,10,1,3,15,4,2,11,6,7,12,0,5,14,9]],

        [[10,0,9,14,6,3,15,5,1,13,12,7,11,4,2,8],
         [13,7,0,9,3,4,6,10,2,8,5,14,12,11,15,1],
         [13,6,4,9,8,15,3,0,11,1,2,12,5,10,14,7],
         [1,10,13,0,6,9,8,7,4,15,14,3,11,5,2,12]],

        [[7,13,14,3,0,6,9,10,1,2,8,5,11,12,4,15],
         [13,8,11,5,6,15,0,3,4,7,2,12,1,10,14,9],
         [10,6,9,0,12,11,7,13,15,1,3,14,5,2,8,4],
         [3,15,0,6,10,1,13,8,9,4,5,11,12,7,2,14]],

        [[2,12,4,1,7,10,11,6,8,5,3,15,13,0,14,9],
         [14,11,2,12,4,7,13,1,5,0,15,10,3,9,8,6],
         [4,2,1,11,10,13,7,8,15,9,12,5,6,3,0,14],
         [11,8,12,7,1,14,2,13,6,15,0,9,10,4,5,3]],

        [[12,1,10,15,9,2,6,8,0,13,3,4,14,7,5,11],
         [10,15,4,2,7,12,9,5,6,1,13,14,0,11,3,8],
         [9,14,15,5,2,8,12,3,7,0,4,10,1,13,11,6],
         [4,3,2,12,9,5,15,10,11,14,1,7,6,0,8,13]],

        [[4,11,2,14,15,0,8,13,3,12,9,7,5,10,6,1],
         [13,0,11,7,4,9,1,10,14,3,5,12,2,15,8,6],
         [1,4,11,13,12,3,7,14,10,15,6,8,0,5,9,2],
         [6,11,13,8,1,4,10,7,9,5,0,15,14,2,3,12]],

        [[13,2,8,4,6,15,11,1,10,9,3,14,5,0,12,7],
         [1,15,13,8,10,3,7,4,12,5,6,11,0,14,9,2],
         [7,11,4,1,9,12,14,2,0,6,10,13,15,3,5,8],
         [2,1,14,7,4,10,8,13,15,12,9,0,3,5,6,11]]
    ]

    result = ""
    for i in range(8):
        block = bits[i*6:(i+1)*6]
        row = int(block[0] + block[-1], 2)
        col = int(block[1:5], 2)
        val = sbox[i][row][col]
        result += bin(val)[2:].zfill(4)
    return result

# DES key schedule and round function
def generate_keys(key_64bit):
    key = hex_to_bin(key_64bit)

    # Initial key permutation table (PC1)
    keyp = [57,49,41,33,25,17,9,
            1,58,50,42,34,26,18,
            10,2,59,51,43,35,27,
            19,11,3,60,52,44,36,
            63,55,47,39,31,23,15,
            7,62,54,46,38,30,22,
            14,6,61,53,45,37,29,
            21,13,5,28,20,12,4]

    # Shift table
    shift_table = [1, 1, 2, 2, 2, 2, 2, 2,
                   1, 2, 2, 2, 2, 2, 2, 1]

    # Key compression table (PC2)
    key_comp = [14,17,11,24,1,5,
                3,28,15,6,21,10,
                23,19,12,4,26,8,
                16,7,27,20,13,2,
                41,52,31,37,47,55,
                30,40,51,45,33,48,
                44,49,39,56,34,53,
                46,42,50,36,29,32]

    key = permute(key, keyp)
    left = key[:28]
    right = key[28:]
    round_keys = []

    for i in range(16):
        left = shift_left(left, shift_table[i])
        right = shift_left(right, shift_table[i])
        combined = left + right
        round_key = permute(combined, key_comp)
        round_keys.append(round_key)

    return round_keys

# Initial and Final Permutation tables
initial_permutation = [58,50,42,34,26,18,10,2,
                       60,52,44,36,28,20,12,4,
                       62,54,46,38,30,22,14,6,
                       64,56,48,40,32,24,16,8,
                       57,49,41,33,25,17,9,1,
                       59,51,43,35,27,19,11,3,
                       61,53,45,37,29,21,13,5,
                       63,55,47,39,31,23,15,7]

final_permutation = [40,8,48,16,56,24,64,32,
                     39,7,47,15,55,23,63,31,
                     38,6,46,14,54,22,62,30,
                     37,5,45,13,53,21,61,29,
                     36,4,44,12,52,20,60,28,
                     35,3,43,11,51,19,59,27,
                     34,2,42,10,50,18,58,26,
                     33,1,41,9,49,17,57,25]

# DES round function
def des_round_function(bits, round_key):
    # Expansion D-box Table
    expansion_table = [32,1,2,3,4,5,4,5,
                       6,7,8,9,8,9,10,11,
                       12,13,12,13,14,15,16,17,
                       16,17,18,19,20,21,20,21,
                       22,23,24,25,24,25,26,27,
                       28,29,28,29,30,31,32,1]

    # Straight Permutation Table
    per = [16,7,20,21,29,12,28,17,
           1,15,23,26,5,18,31,10,
           2,8,24,14,32,27,3,9,
           19,13,30,6,22,11,4,25]

    left = bits[:32]
    right = bits[32:]

    right_expanded = permute(right, expansion_table)
    xored = xor(right_expanded, round_key)
    sboxed = sbox_substitution(xored)
    sboxed_permuted = permute(sboxed, per)
    result = xor(left, sboxed_permuted)

    return right, result

# Encryption
def encrypt(plain_text, round_keys):
    text = hex_to_bin(plain_text)
    text = permute(text, initial_permutation)
    print("After initial permutation", bin_to_hex(text))

    left = text[:32]
    right = text[32:]

    for i in range(16):
        left, right = des_round_function(left + right, round_keys[i])
        combined = left + right
        print("Round ", i + 1, " ", bin_to_hex(left), " ", bin_to_hex(right), " ", bin_to_hex(combined))

    combined = right + left  # note the swap
    cipher_text = permute(combined, final_permutation)
    print("Cipher Text : ", bin_to_hex(cipher_text))
    return cipher_text

# Decryption
def decrypt(cipher_text, round_keys):
    text = permute(cipher_text, initial_permutation)
    print("After initial permutation", bin_to_hex(text))

    left = text[:32]
    right = text[32:]

    for i in range(16):
        left, right = des_round_function(left + right, round_keys[15 - i])
        combined = left + right
        print("Round ", i + 1, " ", bin_to_hex(left), " ", bin_to_hex(right), " ", bin_to_hex(combined))

    combined = right + left
    plain_text = permute(combined, final_permutation)
    print("Plain Text : ", bin_to_hex(plain_text))
    return plain_text

# MAIN
print("Encryption")
key = "AABB09182736CCDD"
plain_text = "123456ABCD132536"

round_keys = generate_keys(key)
cipher_text = encrypt(plain_text, round_keys)

print("Decryption")
decrypt(cipher_text, round_keys)


Encryption
After initial permutation 14A7D67818CA18AD
Round  1   18CA18AD   5A78E394   18CA18AD5A78E394
Round  2   5A78E394   4A1210F6   5A78E3944A1210F6
Round  3   4A1210F6   B8089591   4A1210F6B8089591
Round  4   B8089591   236779C2   B8089591236779C2
Round  5   236779C2   A15A4B87   236779C2A15A4B87
Round  6   A15A4B87   2E8F9C65   A15A4B872E8F9C65
Round  7   2E8F9C65   A9FC20A3   2E8F9C65A9FC20A3
Round  8   A9FC20A3   308BEE97   A9FC20A3308BEE97
Round  9   308BEE97   10AF9D37   308BEE9710AF9D37
Round  10   10AF9D37   6CA6CB20   10AF9D376CA6CB20
Round  11   6CA6CB20   FF3C485F   6CA6CB20FF3C485F
Round  12   FF3C485F   22A5963B   FF3C485F22A5963B
Round  13   22A5963B   387CCDAA   22A5963B387CCDAA
Round  14   387CCDAA   BD2DD2AB   387CCDAABD2DD2AB
Round  15   BD2DD2AB   CF26B472   BD2DD2ABCF26B472
Round  16   CF26B472   19BA9212   CF26B47219BA9212
Cipher Text :  C0B7A8D05F3A829C
Decryption
After initial permutation 19BA9212CF26B472
Round  1   CF26B472   BD2DD2AB   CF26B472BD2DD2AB
Rou

'0001001000110100010101101010101111001101000100110010010100110110'